# Observing the Appearance of the Quantum Zeno Effect 

We use the program [HFSS by Ansys][HFSS], in addition with the [pyEPR python library][pyEPR] to find the parameters of the system.

[HFSS]: https://www.ansys.com/products/electronics/ansys-hfss
[pyEPR]: https://github.com/zlatko-minev/pyEPR

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyEPR as epr

plt.style.use('ggplot')
plt.style.use('seaborn-darkgrid')

#### 🔹 Connecting to HFSS

In [2]:
pinfo = epr.Project_Info(project_path = '.', 
                         project_name = 'QuantumZeno',  # File name
                         design_name  = 'WithPins')

INFO 10:41AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:41AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:41AM [load_ansys_project]: 	Opened Ansys App
INFO 10:41AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:41AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    E:/QuantumZeno_LocalSimulations/
	Project:   QuantumZeno
INFO 10:41AM [connect_design]: 	Opened active design
	Design:    WithPins [Solution type: Eigenmode]
INFO 10:41AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:41AM [connect]: 	Connected to project "QuantumZeno" and design "WithPins" 😀 



#### 🔹 Run setup (Find modes and Q-factors in HFSS)

In [3]:
for var in pinfo.design.get_variable_names():
    print(f'{var} = {pinfo.design.get_variable_value(var)}')
pinfo.setup.analyze()

INFO 10:41AM [analyze]: Analyzing setup Setup1


BrightTransmonXshift = 900um
TransmonLeadWidth = 20um
TransmonLeadLength = 90um
TransmonPadWidth = 290um
TransmonPadLength = 590um
TransmonsDistance = 190um
DarkTransmonPadLength = 560um
DarkTransmonLj = 8nH
BrightTransmonLj = 8nH
DarkTransmonPadWidth = 600um
PinLength = 5.2mm


One can access the simulated results using pyEPR.

In [ ]:
eprh = epr.DistributedAnalysis(pinfo)
df = eprh.get_freqs_bare_pd(eprh.variations[0])
df['Lifetime (s)'] = df['Quality Factor']/(2*np.pi*df['Freq. (GHz)']*1e9)
df

Design "WithPins" info:
	# eigenmodes    3
	# variations    1


,Freq. (GHz),Quality Factor,Lifetime (s)
mode,,,
0,5.382058,1.698788e+06,5.023552e-05
1,6.515107,2.909624e+04,7.107803e-07
2,7.606286,2.501775e+03,5.234748e-08


Here modes 0, 1 and 2 are the transmons and RO resonator modes, respectively.

#### 🔹 Define the junction (and any other non-linear component)

In [ ]:
pinfo.junctions['j1'] = {'Lj_variable' : 'BrightTransmonLj',
                         'rect'        : 'BrightTransmonJJ', 
                         'line'        : 'BrightJJline', 
                         'length'      : epr.parse_units('100um')}

pinfo.junctions['j2'] = {'Lj_variable' : 'DarkTransmonLj',
                         'rect'        : 'DarkTransmonJJ',
                         'line'        : 'DarkJJline',
                         'length'      : epr.parse_units('100um')}

# Check that valid names of variables and objects have been supplied.
# An error is raised with a message if something is wrong.
pinfo.validate_junction_info()

#### 🔹 Distributed Analysis (not quantum)

Note that **3 modes is the maximum we can fully analyze❗** See [this issue][issue]

Notice that the modes numbers one wishes to analyze need to be inserted *manually*. They can be identified by looking at the field plots in HFSS.

[issue]: https://github.com/zlatko-minev/pyEPR/issues/36

In [ ]:
modes = [0, 1, 2]
eprh.do_EPR_analysis(modes=modes)


Variation 0  [1/1]

  Mode 0 at 5.38 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               97.4%  2.203e-24 5.714e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.20%
	j1              0.037965  (+)        0.000694641
		Energy fraction (Lj over Lj&Cj)= 98.20%
	j2              0.935339  (+)        0.0171138
		(U_tot_cap-U_tot_ind)/mean=0.92%

  Mode 1 at 6.52 GHz   [2/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               93.0%  4.088e-25 2.865e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_1j   sign s_1j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.39%
	j1              0.890382  (+)        0.0238726
		Energy fraction (Lj over Lj&Cj)= 97.39%
	j2            

(WindowsPath('C:/data-pyEPR/QuantumZeno/WithPins/2021-08-10 16-05-05.npz'),
 ['0'])

#### 🔹 Quantum Analysis (yes quantum)

In [ ]:
epra = epr.QuantumAnalysis(eprh.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 15)

WARNING 04:05PM [__init__]: <p>Error: <class 'IndexError'></p>


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.018830
1    1.026062
2    1.019120
dtype: float64

Pm_norm idx =
      j1     j2
0  False   True
1   True  False
2  False  False
*** P (participation matrix, not normlz.)
         j1        j2
0  0.037301  0.918974
1  0.868726  0.038417
2  0.040189  0.000255

*** S (sign-bit matrix)
   s_j1  s_j2
0     1     1
1     1     1
2     1     1
*** P (participation matrix, normalized.)
     0.037     0.94
      0.89    0.038
      0.04  0.00026

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       156     29.7    0.871
      29.7      207     21.7
     0.871     21.7    0.572

*** Chi matrix ND (MHz) 
       184     51.1     0.72
      51.1      214     13.2
      0.72     13.2    0.252

*** Frequencies O1 PT (MHz)
0    5211.185779
1    6282.693133
2   

OrderedDict([('0',
              OrderedDict([('f_0', 0    5382.058204
                            1    6515.107460
                            2    7606.285631
                            Name: 0, dtype: float64),
                           ('f_1',
                            0    5211.185779
                            1    6282.693133
                            2    7594.415388
                            dtype: float64),
                           ('f_ND',
                            0    5196.784170
                            1    6275.589144
                            2    7596.160382
                            dtype: float64),
                           ('chi_O1',
                                        0           1          2
                            0  155.589070   29.696181   0.870529
                            1   29.696181  206.702950  21.726574
                            2    0.870529   21.726574   0.571691),
                           ('chi_ND',
                

In [ ]:
# elements_names = ['bright', 'dark', 'readout']
elements_names = ['dark', 'bright', 'readout']
df2 = epra.get_frequencies(numeric=True) 
df2.columns = ['Freq. ND (MHz)']
df2['Quality Factor'] = eprh.get_freqs_bare_pd(eprh.variations[0]).loc[modes]['Quality Factor']
df2['Lifetime (s)'] = df2['Quality Factor']/(2*np.pi*df2['Freq. ND (MHz)']*1e6)
df2.index = elements_names
display(df2)

,Freq. ND (MHz),Quality Factor,Lifetime (s)
bright,5196.784170,1.698788e+06,5.202650e-05
dark,6275.589144,2.909624e+04,7.379084e-07
readout,7596.160382,2.501775e+03,5.241725e-08


### Cross-kerr and anharmonicities

pyEPR just calculated for us all the elements of $\chi_{ij}$ ($3\times 3$ symmetric matrix).
The diagonal elements of the matrix are the anharmonicities $\alpha_i$ and the off-diagonal elements are the coupling between the modes.

In [ ]:
chis = epra.get_chis()

# anharmonicities
alphas = np.diag(chis)
alphas_df = pd.DataFrame({'Anharmonicity (MHz)': alphas}, index=elements_names)
display(alphas_df)

# cross-Kerr
chis = [chis[i][j] for i in range(len(elements_names)) for j in range(len(elements_names)) if i<j]
index = [f'{elements_names[i]}-{elements_names[j]}' for i in range(len(elements_names)) for j in range(len(elements_names)) if i<j]
chis_df = pd.DataFrame({'Coupling (MHz)': chis}, index=index)
display(chis_df)

,Anharmonicity (MHz)
bright,183.792059
dark,214.319960
readout,0.252360


,Coupling (MHz)
bright-dark,51.141465
bright-readout,0.719625
dark-readout,13.240475


In [ ]:
# chis_df.to_clipboard(excel=False)